In [25]:
import urllib.request
import urllib.parse
import re
import datetime
import csv
import http.cookiejar
import time

# Function to fetch the CSRF token from the main page
def get_csrf_token_and_cookies():
    url = 'https://www.sharesansar.com/today-share-price'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Mobile Safari/537.36',
    }
    req = urllib.request.Request(url, headers=headers)
    
    # Use a cookie jar to store cookies
    cookie_jar = http.cookiejar.CookieJar()
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cookie_jar))
    response = opener.open(req)
    html = response.read().decode('utf-8')
    
    # Extract CSRF token from the hidden input field
    token_pattern = r'<input type="hidden" name="_token" value="([^"]+)"'
    token = re.search(token_pattern, html).group(1)
    
    return token, cookie_jar


In [27]:
# Function to perform a POST request and fetch HTML content
def fetch_html(date, token, cookie_jar):
    url = 'https://www.sharesansar.com/ajaxtodayshareprice'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Mobile Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    }
    data = urllib.parse.urlencode({'date': date, '_token': token}).encode('utf-8')
    req = urllib.request.Request(url, data=data, headers=headers)
    time.sleep(2)
    
    # Use the same cookie jar for the request
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cookie_jar))
    time.sleep(2)

    response = opener.open(req)
    time.sleep(2)

    return response.read().decode('utf-8')

In [28]:
# Function to extract share prices from HTML content
def parse_share_prices(html_content, date):
    # Extract the table rows
    tbody_pattern = r'<tbody>(.*?)</tbody>'
    tbody_match = re.search(tbody_pattern, html_content, re.DOTALL)
    
    if not tbody_match:
        return []
    
    tbody_content = tbody_match.group(1)
    row_pattern = r'<tr>(.*?)</tr>'
    rows = re.findall(row_pattern, tbody_content, re.DOTALL)
    
    data = []
    for row in rows:
        cols_pattern = r'<td.*?>(.*?)</td>'
        cols = re.findall(cols_pattern, row, re.DOTALL)
        cols = [re.sub(r'<.*?>', '', col).strip() for col in cols]  # Remove HTML tags and strip whitespace
        if len(cols) == 21:
            data.append([date] + cols)
    
    return data

In [29]:
# Function to generate dates between two date ranges
def date_range(start_date, end_date):
    current_date = start_date
    while current_date <= end_date:
        yield current_date
        current_date += datetime.timedelta(days=1)

In [30]:
start_date = datetime.date(2024, 1, 1)
end_date = datetime.date(2024, 1, 3)
csv_filename = "nepse.csv"

# Fetch CSRF token and initial cookies
token, cookie_jar = get_csrf_token_and_cookies()

In [31]:
print(token)
print(cookie_jar)

kvlfgqt21TQQBldd3xt1TtebP3mmJwpyX6Eum6HZ
<CookieJar[<Cookie XSRF-TOKEN=eyJpdiI6IkNGaHp1NTdabEcxeVZ0aXM4VjVKSUE9PSIsInZhbHVlIjoiVThVbS9Dbkx4Z3p5bFFwTXdFZTFIK0R6dGdFQ1duOWxjSFRCQjNIOEJPcHNLUTVTVmt4cks2Nks0Ykd2cFkrNFFPZm85MkpKUFoxR2IxS1NENHNHRjkxcUJudmJBS2JLQ2kzZ3BCdzluR2djSnA4cEh0V2NYTnRCa0lpRHF6ZGwiLCJtYWMiOiI0YjM2ZGY4NTM3ZTU2M2Q5NThhOGM3ZGZmYzZlODQwMmMxY2UxYzIzZTM3ZDdlNTBhZmQ1MGJiYmM5Yjk4ZjA5In0%3D for www.sharesansar.com/>, <Cookie sharesansar_session=eyJpdiI6IjUwSmQrQU5GWC9hQjlDeDBjM2I5M1E9PSIsInZhbHVlIjoidDhOdWhvRS9pK3JQWXN0cExYYnd0Tmt0OUZnSUxOZUhNS0dEaGUvbWVGTWxVZXVhODhIK1EvTG5WWCt4bFJmOERUSHVUUUhHeUNyZUdmN09odXA3TndJYUpLaUJPbVNOeVpUQmRzSHBDczZrT01sS2RLdFVJU1plN2RCSWJRdFciLCJtYWMiOiI4ZmMwY2MxNTkyZjVjN2IyZDI3NzM3MmNmOGVmMjdkZTE2MGZhNmJmMzYyNDUyNzVlNzQ4YmQyMzUxN2E2MmMwIn0%3D for www.sharesansar.com/>]>


In [32]:
# with open(csv_filename, 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerow(['Date', 'S.No', 'Symbol', 'Conf', 'Open', 'High', 'Low', 'Close', 'VWAP', 'Vol', 'Prev. Close', 'Turnover', 'Trans', 'Diff', 'Range', 'Diff %', 'Range %', 'VWAP %', '120 Days', '180 Days', '52 Weeks High', '52 Weeks Low'])

#     for single_date in date_range(start_date, end_date):
#         formatted_date = single_date.strftime('%Y-%m-%d')
#         try:
#             html_content = fetch_html(formatted_date, token, cookie_jar)
#             share_prices = parse_share_prices(html_content, formatted_date)
            
#             if share_prices:
#                 for row in share_prices:
#                     writer.writerow(row)
#                 print(f"Data for {formatted_date} written to CSV.")
#             else:
#                 print(f"No data for {formatted_date}.")
            
#         except Exception as e:
#             print(f"Error fetching data for {formatted_date}: {e}")

# print("CSV file exported successfully.")